## Using Python to Perform Extract-Transform-Load (ETL Processing) + SQL Query

Interact with the RDBMS from a remote client running Python. Fetch data into Pandas DataFrames, perform all the necessary transformations in-memory on the client, and then push the newly transformed DataFrame back to the RDBMS.

#### Import the Necessary Libraries

In [1]:
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine

#### Declare & Assign Connection Variables for the MySQL Server & Databases with which You'll be Working 

In [2]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "Scout1210!"

src_dbname = "chinook"
dst_dbname = "chinook_dw"

#### Define Functions for Getting Data From and Setting Data Into Databases

In [3]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

#### Create the New Data Warehouse database, and to Use it, Switch the Connection Context.

In [4]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

### Create & Populate the Dimension Tables
#### Extract Data from the Source Database Tables

In [5]:
sql_customers = "SELECT * FROM chinook.Customer;"

try:
    df_customers = get_dataframe(user_id, pwd, host_name, src_dbname, sql_customers)
except:
    print ("Error: unable to fetch data")
    
df_customers.head(2)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5


In [6]:
sql_employees = "SELECT * FROM chinook.employee;"

try:
    df_employees = get_dataframe(user_id, pwd, host_name, src_dbname, sql_employees)
except:
    print ("Error: unable to fetch data")

df_employees.head(2)

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18,2002-08-14,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08,2002-05-01,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com


#### Perform Any Necessary Transformations
drop features that don't provide any real value to the analytics solution. Then, rename the primary key column to conform with data warehouse design standards.

In [7]:
#Explore df_customers
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerId    59 non-null     int64 
 1   FirstName     59 non-null     object
 2   LastName      59 non-null     object
 3   Company       10 non-null     object
 4   Address       59 non-null     object
 5   City          59 non-null     object
 6   State         30 non-null     object
 7   Country       59 non-null     object
 8   PostalCode    55 non-null     object
 9   Phone         58 non-null     object
 10  Fax           12 non-null     object
 11  Email         59 non-null     object
 12  SupportRepId  59 non-null     int64 
dtypes: int64(2), object(11)
memory usage: 6.1+ KB


In [8]:
#Email becuase not needed and Fax because high number of nulls
drop_cols = ['Email','Fax']
df_customers.drop(drop_cols, axis=1, inplace=True)

#Rename Id, to Key. Also rename SupportRepId to employee_key to match future tables
df_customers.rename(columns={"CustomerId":"customer_key", "SupportRepId": "employee_key"}, inplace=True)

df_customers.head(2)

,customer_key,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,employee_key
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,5


In [9]:
#Explore df_employees
df_employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   EmployeeId  8 non-null      int64         
 1   LastName    8 non-null      object        
 2   FirstName   8 non-null      object        
 3   Title       8 non-null      object        
 4   ReportsTo   7 non-null      float64       
 5   BirthDate   8 non-null      datetime64[ns]
 6   HireDate    8 non-null      datetime64[ns]
 7   Address     8 non-null      object        
 8   City        8 non-null      object        
 9   State       8 non-null      object        
 10  Country     8 non-null      object        
 11  PostalCode  8 non-null      object        
 12  Phone       8 non-null      object        
 13  Fax         8 non-null      object        
 14  Email       8 non-null      object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(11)
memory usage: 1.1+ KB


In [10]:
#Email, Fax, Birth date, and hire date not needed, Drop them
drop_cols = ['Email','Fax', 'BirthDate', 'HireDate']
df_employees.drop(drop_cols, axis=1, inplace=True)

#Rename Id to Key
df_employees.rename(columns={"EmployeeId":"employee_key"}, inplace=True)

df_employees.head(2)

,employee_key,LastName,FirstName,Title,ReportsTo,Address,City,State,Country,PostalCode,Phone
0,1,Adams,Andrew,General Manager,NaN,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482
1,2,Edwards,Nancy,Sales Manager,1.0,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443


#### Load the Transformed DataFrames into the New Data Warehouse by Creating New Tables

In [11]:
db_operation = "insert"

tables = [('dim_customers', df_customers, 'customer_key'),
          ('dim_employees', df_employees, 'employee_key')]

In [12]:
for table_name, dataframe, primary_key in tables:
    try:
        set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)
    except:
        print ("Error: unable to send data: " + table_name)

### 2.0. Create & Populate the Fact Table

Query the source **chinook** database to fill a *dataframe* for each of the source tables we need to create our *fact_invoices* fact table; invoice, invoiceline. Then *join* those *dataframes* using the **merge( )** method of the Pandas DataFrame, make any additional changes needed, and push the *dataframe* back to the MySQL server to create and populate the new *fact* table.

##### Get all the data from each of the four tables involved

In [13]:
invoices_sql = "SELECT * FROM chinook.Invoice;"

try:
    df_invoices = get_dataframe(user_id, pwd, host_name, src_dbname, invoices_sql)
except:
    print ("Error: unable to fetch data")

#no need to rename anything yet, already unique id name (InvoiceId)
df_invoices.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11,69 Salem Street,Boston,MA,USA,2113,13.86


In [14]:
invoice_details_sql = "SELECT * FROM chinook.InvoiceLine;"

try:
    df_invoice_details = get_dataframe(user_id, pwd, host_name, src_dbname, invoice_details_sql)
except:
    print ("Error: unable to fetch data")

#no need to rename yet, already unique id name (InvoiceLineId)
df_invoice_details.head()

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1


In [15]:
#Explore df_invoices
df_invoices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 412 entries, 0 to 411
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   InvoiceId          412 non-null    int64         
 1   CustomerId         412 non-null    int64         
 2   InvoiceDate        412 non-null    datetime64[ns]
 3   BillingAddress     412 non-null    object        
 4   BillingCity        412 non-null    object        
 5   BillingState       210 non-null    object        
 6   BillingCountry     412 non-null    object        
 7   BillingPostalCode  384 non-null    object        
 8   Total              412 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 29.1+ KB


In [16]:
#explore df_invoice_details
df_invoice_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   InvoiceLineId  2240 non-null   int64  
 1   InvoiceId      2240 non-null   int64  
 2   TrackId        2240 non-null   int64  
 3   UnitPrice      2240 non-null   float64
 4   Quantity       2240 non-null   int64  
dtypes: float64(1), int64(4)
memory usage: 87.6 KB


Since each **Invoice** (the *left* dataframe) can have many **Invoice details** (the *right* dataframe), we'll need to implement a **right** *outer join* **on** the *InvoiceId* column.

In [17]:
#Join the two dataframes on InvoiceId
df_fact_invoices = pd.merge(df_invoices, df_invoice_details, on='InvoiceId', how='right')
df_fact_invoices.head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,InvoiceLineId,TrackId,UnitPrice,Quantity
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,1,2,0.99,1
1,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,2,4,0.99,1
2,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,3,6,0.99,1
3,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,4,8,0.99,1
4,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,5,10,0.99,1


In [18]:
#check to see if right number of rows, 
# invoice_details had one to many relationship to invoice
#should have same number of rows as invoice_details then (2240)

df_fact_invoices.info() #good!

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2240 entries, 0 to 2239
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   InvoiceId          2240 non-null   int64         
 1   CustomerId         2240 non-null   int64         
 2   InvoiceDate        2240 non-null   datetime64[ns]
 3   BillingAddress     2240 non-null   object        
 4   BillingCity        2240 non-null   object        
 5   BillingState       1140 non-null   object        
 6   BillingCountry     2240 non-null   object        
 7   BillingPostalCode  2088 non-null   object        
 8   Total              2240 non-null   float64       
 9   InvoiceLineId      2240 non-null   int64         
 10  TrackId            2240 non-null   int64         
 11  UnitPrice          2240 non-null   float64       
 12  Quantity           2240 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(5), object(5)
memory us

##### Perform any Additional Transformations

In [19]:
#Rename Id's to keys to match existing tables, no confusion in future queries!
df_fact_invoices.rename(columns={"InvoiceId":"invoice_key", "CustomerId":"customer_key","TrackId":"track_key"}, inplace=True)

#drop unnecessary columns, ex: invoicelineid since no need for foreign reference since in same dataframe now
df_fact_invoices.drop(['InvoiceLineId'], axis=1, inplace=True)
df_fact_invoices.head()

,invoice_key,customer_key,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total,track_key,UnitPrice,Quantity
0,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,2,0.99,1
1,1,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98,4,0.99,1
2,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,6,0.99,1
3,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,8,0.99,1
4,2,4,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,3.96,10,0.99,1


In [20]:
# Reorder the columns
ordered_columns = ['invoice_key','customer_key','track_key','InvoiceDate','BillingAddress','BillingCity',
                   'BillingState','BillingCountry','BillingPostalCode','UnitPrice',
                   'Quantity', 'Total']
df_fact_invoices = df_fact_invoices[ordered_columns]

In [21]:
# Insert new column with auto-incremented numeric value as the primary key.
df_fact_invoices.insert(0, "fact_invoice_key", range(1, df_fact_invoices.shape[0]+1))
df_fact_invoices.head(5)

,fact_invoice_key,invoice_key,customer_key,track_key,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,UnitPrice,Quantity,Total
0,1,1,2,2,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1,1.98
1,2,1,2,4,2009-01-01,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,0.99,1,1.98
2,3,2,4,6,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,0.99,1,3.96
3,4,2,4,8,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,0.99,1,3.96
4,5,2,4,10,2009-01-02,Ullevålsveien 14,Oslo,None,Norway,0171,0.99,1,3.96


##### Write the DataFrame Back to the Database

In [22]:
table_name = "fact_invoices"
primary_key = "fact_invoice_key"
db_operation = "insert"

try:
        set_dataframe(user_id, pwd, host_name, dst_dbname, df_fact_invoices, table_name, primary_key, db_operation)
except:
        print ("Error: unable to send data")

### Demonstrate that the New Data Warehouse Exists and Contains the Correct Data
To demonstrate the viability of the solution, Here is a SQL SELECT statement that returns:
-	Each Customer’s Last Name (customers table)
-   First Name of employee that helped them (employee table)
-	The total amount of the order quantity associated with each customer (fact_invoices table)
-	The total amount of the order unit price associated with each customer (fact_invoices table)

In [23]:
sql2 = """
    SELECT c.lastName AS Custmer_name, c.customer_key, e.FirstName AS Employee_Name, e.employee_key, SUM(f.quantity) AS total_quantity, SUM(f.unitprice) AS total_price
    FROM chinook_dw.dim_customers as c
    LEFT OUTER JOIN chinook_dw.dim_employees as e
    On c.employee_key = e.employee_key
    LEFT OUTER JOIN chinook_dw.fact_invoices as f
    On c.customer_key = f.customer_key
    GROUP BY c.lastname
    ORDER BY total_price DESC;
""".format(dst_dbname)

df_test = get_dataframe(user_id, pwd, host_name, src_dbname, sql2)

In [24]:
df_test.head()

,Custmer_name,customer_key,Employee_Name,employee_key,total_quantity,total_price
0,Holý,6,Steve,5,38.0,49.62
1,Cunningham,26,Margaret,4,38.0,47.62
2,Rojas,57,Steve,5,38.0,46.62
3,O'Reilly,46,Jane,3,38.0,45.62
4,Kovács,45,Jane,3,38.0,45.62


In [25]:
#check 
customer_name = df_customers.loc[df_customers['customer_key'] ==6].LastName.values[0]
employee_name = df_employees.loc[df_employees['employee_key'] == 5].FirstName.values[0]

print('customer name: ' + customer_name + ', employee name: ' + employee_name +', total amount order $'  
+str(sum(df_fact_invoices.loc[df_fact_invoices['customer_key'] == 6].UnitPrice))) #same!


customer name: Holý, employee name: Steve, total amount order $49.620000000000005
